In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from google.colab import files
import numpy as np
import pandas as pd
from tensorflow import keras
import os
import io
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

     |████████████████████████████████| 3.3MB 16.8MB/s 
Version:  2.2.0-rc1
Eager mode:  True
Hub version:  0.7.0
GPU is available


In [0]:

uploaded = files.upload()

Saving uglywords.csv to uglywords.csv


In [0]:
data = pd.read_csv(io.BytesIO(uploaded['uglywords.csv']), encoding = 'latin1')

In [0]:
data.head(10)

,Label,Tweet
0,1.0,*screams in 25 different languages*
1,0.0,Families to sue over Legionnaires: More than 4...
2,0.0,Pandemonium In Aba As Woman Delivers Baby With...
3,0.0,My emotions are a train wreck. My body is a tr...
4,1.0,Alton brown just did a livestream and he burne...
5,1.0,@TinyJecht Are you another Stand-user? If you ...
6,1.0,brooke just face timed me at the concert and j...
7,0.0,A group of Florida Forest Service firefighters...
8,0.0,70 Years After Atomic Bombs Japan Still Strugg...
9,0.0,The majority of those killed were civilians on...


In [0]:
data.shape

(739, 2)

In [0]:
list(data.columns.values)

['Label', 'Tweet']

In [0]:
#lets see the count by type of sentiments

print((data.Label.value_counts()/sum(data.Label.value_counts()))*100)

0.0    63.821138
1.0    36.178862
Name: Label, dtype: float64


In [0]:
#Tokenize the text
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
import nltk
nltk.download('wordnet')
import nltk
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# define a function to use to clean the data
def normalizer(tweet):
    no_urls = re.sub(r"http\S+", " " ,tweet)
    only_letters = re.sub("[^a-zA-Z]", " ",no_urls)
    tokens = nltk.word_tokenize(only_letters)[2 :]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

In [0]:
normalizer('Shamiso is so dangerous because you never know what her emotions mean')

['dangerous', 'never', 'know', 'emotion', 'mean']

In [0]:
#apply the function to clean the data:
pd.set_option('display.max_colwidth', -1)
data['normalized_tweet'] = data.Tweet.apply(normalizer)
data[['Tweet', 'normalized_tweet']].head(10)

,Tweet,normalized_tweet
0,*screams in 25 different languages*,"[different, language]"
1,Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/ZA4AXFJSVB,"[sue, legionnaire, family, affected, fatal, outbreak, legionnaire, disea]"
2,Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) - http://t.co/c5u9qshhnb,"[aba, woman, delivers, baby, without, face, photo]"
3,My emotions are a train wreck. My body is a train wreck. I'm a wreck,"[train, wreck, body, train, wreck, wreck]"
4,Alton brown just did a livestream and he burned the butter and touched the hot plate too soon and made a nut joke http://t.co/gvd7fcx8iZ,"[livestream, burned, butter, touched, hot, plate, soon, made, nut, joke]"
5,@TinyJecht Are you another Stand-user? If you are I will have to detonate you with my Killer Queen.,"[another, stand, user, detonate, killer, queen]"
6,brooke just face timed me at the concert and just screamed for 2 minutes straight,"[face, timed, concert, screamed, minute, straight]"
7,A group of Florida Forest Service firefighters could be deployed to California to help contain fires. Details at 10! http://t.co/fwuP9YURzY,"[florida, forest, service, firefighter, could, deployed, california, help, contain, fire, detail]"
8,70 Years After Atomic Bombs Japan Still Struggles With War Past: The anniversary of the devastation wrought b... http://t.co/iTBJ6DKRZI,"[atomic, bomb, japan, still, struggle, war, past, anniversary, devastation, wrought, b]"
9,The majority of those killed were civilians on the ground after the jet first bombed the city's main street then dramatically plummeted,"[killed, civilian, ground, jet, first, bombed, city, main, street, dramatically, plummeted]"


In [0]:
test = 'https://www.shatorangola.com is the best company where you can buy all your PPE and get help with visual inspection whenever you need it%?//267'

In [0]:
print(normalizer(test))

['best', 'company', 'buy', 'ppe', 'get', 'help', 'visual', 'inspection', 'whenever', 'need']


In [0]:
data.head()

,Label,Tweet,normalized_tweet
0,1.0,*screams in 25 different languages*,"[different, language]"
1,0.0,Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/ZA4AXFJSVB,"[sue, legionnaire, family, affected, fatal, outbreak, legionnaire, disea]"
2,0.0,Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) - http://t.co/c5u9qshhnb,"[aba, woman, delivers, baby, without, face, photo]"
3,0.0,My emotions are a train wreck. My body is a train wreck. I'm a wreck,"[train, wreck, body, train, wreck, wreck]"
4,1.0,Alton brown just did a livestream and he burned the butter and touched the hot plate too soon and made a nut joke http://t.co/gvd7fcx8iZ,"[livestream, burned, butter, touched, hot, plate, soon, made, nut, joke]"


In [0]:
#Now we shall look at the words that contributed to each type of sentiment more than the other

In [0]:
from nltk import ngrams
def ngrams(input_list):
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams + trigrams
data['grams'] = data.normalized_tweet.apply(ngrams)
data[['grams']].head()

,grams
0,[different language]
1,"[sue legionnaire, legionnaire family, family affected, affected fatal, fatal outbreak, outbreak legionnaire, legionnaire disea, sue legionnaire family, legionnaire family affected, family affected fatal, affected fatal outbreak, fatal outbreak legionnaire, outbreak legionnaire disea]"
2,"[aba woman, woman delivers, delivers baby, baby without, without face, face photo, aba woman delivers, woman delivers baby, delivers baby without, baby without face, without face photo]"
3,"[train wreck, wreck body, body train, train wreck, wreck wreck, train wreck body, wreck body train, body train wreck, train wreck wreck]"
4,"[livestream burned, burned butter, butter touched, touched hot, hot plate, plate soon, soon made, made nut, nut joke, livestream burned butter, burned butter touched, butter touched hot, touched hot plate, hot plate soon, plate soon made, soon made nut, made nut joke]"


In [0]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

In [0]:
#word counts per label in from high to low
data[(data.Label == 1)][['grams']].apply(count_words)['grams'].most_common(15)

[('gon na', 4),
 ('forest fire', 3),
 ('full read', 3),
 ('body bag', 2),
 ('year old', 2),
 ('anybody else', 2),
 ('read ebay', 2),
 ('full read ebay', 2),
 ('issue issue', 2),
 ('syrian refugee', 2),
 ('quarantine offensive', 2),
 ('offensive content', 2),
 ('quarantine offensive content', 2),
 ('high school', 2),
 ('meat loving', 2)]

In [0]:
#word counts per label in from high to low
data[(data.Label == 0)][['grams']].apply(count_words)['grams'].most_common(15)

[('airport get', 5),
 ('get swallowed', 5),
 ('swallowed sandstorm', 5),
 ('sandstorm minute', 5),
 ('airport get swallowed', 5),
 ('get swallowed sandstorm', 5),
 ('swallowed sandstorm minute', 5),
 ('oil spill', 5),
 ('mass murder', 4),
 ('northern california', 4),
 ('disaster typhoon', 4),
 ('typhoon devastated', 4),
 ('devastated saipan', 4),
 ('disaster typhoon devastated', 4),
 ('typhoon devastated saipan', 4)]

In [0]:
#data = data.drop(['grams'], axis=1)
data = data.drop(['Tweet'], axis=1)

In [0]:
data.to_csv('clean_text.csv')

In [0]:
#split dataset

In [0]:
trained_dataset_url = "/content/clean_text.csv"

In [0]:
df_train = pd.read_csv(trained_dataset_url)

In [0]:
df_train.head()

,Unnamed: 0,Label,normalized_tweet,grams
0,0,1.0,"['different', 'language']",['different language']
1,1,0.0,"['sue', 'legionnaire', 'family', 'affected', 'fatal', 'outbreak', 'legionnaire', 'disea']","['sue legionnaire', 'legionnaire family', 'family affected', 'affected fatal', 'fatal outbreak', 'outbreak legionnaire', 'legionnaire disea', 'sue legionnaire family', 'legionnaire family affected', 'family affected fatal', 'affected fatal outbreak', 'fatal outbreak legionnaire', 'outbreak legionnaire disea']"
2,2,0.0,"['aba', 'woman', 'delivers', 'baby', 'without', 'face', 'photo']","['aba woman', 'woman delivers', 'delivers baby', 'baby without', 'without face', 'face photo', 'aba woman delivers', 'woman delivers baby', 'delivers baby without', 'baby without face', 'without face photo']"
3,3,0.0,"['train', 'wreck', 'body', 'train', 'wreck', 'wreck']","['train wreck', 'wreck body', 'body train', 'train wreck', 'wreck wreck', 'train wreck body', 'wreck body train', 'body train wreck', 'train wreck wreck']"
4,4,1.0,"['livestream', 'burned', 'butter', 'touched', 'hot', 'plate', 'soon', 'made', 'nut', 'joke']","['livestream burned', 'burned butter', 'butter touched', 'touched hot', 'hot plate', 'plate soon', 'soon made', 'made nut', 'nut joke', 'livestream burned butter', 'burned butter touched', 'butter touched hot', 'touched hot plate', 'hot plate soon', 'plate soon made', 'soon made nut', 'made nut joke']"


In [0]:
df_train=df_train.drop(['Unnamed: 0'], axis=1)

In [0]:
df_train.head()

,Label,normalized_tweet,grams
0,1.0,"['different', 'language']",['different language']
1,0.0,"['sue', 'legionnaire', 'family', 'affected', 'fatal', 'outbreak', 'legionnaire', 'disea']","['sue legionnaire', 'legionnaire family', 'family affected', 'affected fatal', 'fatal outbreak', 'outbreak legionnaire', 'legionnaire disea', 'sue legionnaire family', 'legionnaire family affected', 'family affected fatal', 'affected fatal outbreak', 'fatal outbreak legionnaire', 'outbreak legionnaire disea']"
2,0.0,"['aba', 'woman', 'delivers', 'baby', 'without', 'face', 'photo']","['aba woman', 'woman delivers', 'delivers baby', 'baby without', 'without face', 'face photo', 'aba woman delivers', 'woman delivers baby', 'delivers baby without', 'baby without face', 'without face photo']"
3,0.0,"['train', 'wreck', 'body', 'train', 'wreck', 'wreck']","['train wreck', 'wreck body', 'body train', 'train wreck', 'wreck wreck', 'train wreck body', 'wreck body train', 'body train wreck', 'train wreck wreck']"
4,1.0,"['livestream', 'burned', 'butter', 'touched', 'hot', 'plate', 'soon', 'made', 'nut', 'joke']","['livestream burned', 'burned butter', 'butter touched', 'touched hot', 'hot plate', 'plate soon', 'soon made', 'made nut', 'nut joke', 'livestream burned butter', 'burned butter touched', 'butter touched hot', 'touched hot plate', 'hot plate soon', 'plate soon made', 'soon made nut', 'made nut joke']"


In [0]:
train_data = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df_train.iloc[:500, 1].values, tf.string),
            tf.cast(df_train.iloc[:500, 0].values, tf.int32)
        )
    )
    
)

validation_data = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df_train.iloc[500:590, 1].values, tf.string),
            tf.cast(df_train.iloc[500:590,0].values, tf.int32)
        )
    )
)

In [0]:
from tensorflow import keras

In [0]:
#split labels from data
data_labels = data.Label
data_text = data.normalized_tweet

In [0]:
## Embendings 

In [0]:
embendding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embendding, input_shape =[],
                           dtype=tf.string, trainable = True)

In [0]:
from keras.layers import Dropout
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(15, activation='tanh'))
model.add(tf.keras.layers.Dense(40, activation='tanh'))
model.add(tf.keras.layers.Dense(1))

Using TensorFlow backend.


NameError: ignored

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_7 (Dense)              (None, 40)                640       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 41        
Total params: 401,016
Trainable params: 401,016
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [0]:
history = model.fit(train_data.shuffle(100).batch(48), epochs = 25, validation_data=validation_data.batch(24), verbose = 1)

Epoch 1/25
11/11 [==============================] - 0s 12ms/step - loss: 0.6173 - accuracy: 0.6980 - val_loss: 0.5875 - val_accuracy: 0.7333
Epoch 2/25
11/11 [==============================] - 0s 5ms/step - loss: 0.5095 - accuracy: 0.7160 - val_loss: 0.5434 - val_accuracy: 0.7333
Epoch 3/25
11/11 [==============================] - 0s 4ms/step - loss: 0.4483 - accuracy: 0.7620 - val_loss: 0.5262 - val_accuracy: 0.7444
Epoch 4/25
11/11 [==============================] - 0s 4ms/step - loss: 0.3921 - accuracy: 0.7940 - val_loss: 0.5208 - val_accuracy: 0.7667
Epoch 5/25
11/11 [==============================] - 0s 5ms/step - loss: 0.3480 - accuracy: 0.8280 - val_loss: 0.5230 - val_accuracy: 0.7778
Epoch 6/25
11/11 [==============================] - 0s 4ms/step - loss: 0.3028 - accuracy: 0.8620 - val_loss: 0.5288 - val_accuracy: 0.7778
Epoch 7/25
11/11 [==============================] - 0s 4ms/step - loss: 0.2644 - accuracy: 0.8820 - val_loss: 0.5374 - val_accuracy: 0.7778
Epoch 8/25
11/11 [=

In [0]:
model.predict(["Almeida was very helpful this week. All musicians must give back to society"])

array([[3.5357647]], dtype=float32)

In [0]:
model.predict(["There was a disaster in town. Corona virus has turned the police into violent creatures."])

array([[-2.5859678]], dtype=float32)

In [0]:
model.predict(["The girl was so beautiful and great at everything she touched. May her soul rest in peace."])

array([[3.8291593]], dtype=float32)

In [0]:
model.predict(["This speaker had a great and inspiring academic profile."])

array([[6.369535]], dtype=float32)

In [0]:
model.predict(["There is no toilet paper anywhere in the shops."])

array([[-1.6215246]], dtype=float32)